In [2]:
!wget https://raw.githubusercontent.com/abhilas0/twitter_sentimental_analysis/master/tweet_result.csv

--2021-12-10 05:14:04--  https://raw.githubusercontent.com/abhilas0/twitter_sentimental_analysis/master/tweet_result.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 290015 (283K) [text/plain]
Saving to: ‘tweet_result.csv’

tweet_result.csv    100%[===================>] 283.22K  --.-KB/s    in 0.02s   

2021-12-10 05:14:05 (12.0 MB/s) - ‘tweet_result.csv’ saved [290015/290015]



In [3]:
!wget https://raw.githubusercontent.com/HurmetNoka/religion_on_twitter/master/full_data.csv

--2021-12-10 05:14:05--  https://raw.githubusercontent.com/HurmetNoka/religion_on_twitter/master/full_data.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 9177326 (8.8M) [text/plain]
Saving to: ‘full_data.csv’

full_data.csv       100%[===================>]   8.75M  --.-KB/s    in 0.07s   

2021-12-10 05:14:05 (124 MB/s) - ‘full_data.csv’ saved [9177326/9177326]



In [1]:
import pandas as pd
import numpy as np
import re

politics = pd.read_csv('tweet_result.csv', names = ['date', 'tweet'])
# politics[1].apply()
def remove_pattern(input_txt, pattern):
    r = re.findall(pattern, input_txt)
    for i in r:
        input_txt = re.sub(i, '', input_txt)
        
    return input_txt
politics['tweet'] = np.vectorize(remove_pattern)(politics['tweet'], "@[\w]*")
politics['tweet'] = politics['tweet'].str.replace("[^a-zA-Z#]", " ")
politics.tweet = politics.tweet.apply(lambda x:x[2:])
politics = politics.drop(['date'], axis=1)
politics['label'] = [1 for i in range(len(politics))]
politics

,tweet,label
0,RT Our farmers are our pride amp strength ...,1
1,RT Tej Bahadur is looking all set to become ...,1
2,RT Which law made by PM Modi says Tribals c...,1
3,RT Ramayana stamps released by PM Modi RT ht...,1
4,RT Republic TV falsely portrays man praising...,1
...,...,...
1672,RT AAP Candidate says Vote for gundas to def...,1
1673,India s Narendra Modi has failed at economic r...,1
1674,RT Tej Bahadur is looking all set to become ...,1
1675,Modi effect getting kicked around by Paki...,1


In [2]:
religion = pd.read_csv('full_data.csv') 
religion['tweet'] = religion['tweet_text']
religion = religion.drop(['Unnamed: 0',	'tweet_text'], axis = 1)
religion = religion.sample(n = 1677, random_state=42)
religion.reset_index(drop = True, inplace=True)
religion['label'] = [0 for i in range(len(religion))]
religion

,tweet,label
0,"@ Emmy4Mayim Sadly, that's not true. Historica...",0
1,"TOMORROW, Tuesday, May 31st, at 8 p.m, is anot...",0
2,Islam is the only religion that motivates you ...,0
3,Woe to anyone who believes the self serving ‘r...,0
4,and the reason there are so many different rel...,0
...,...,...
1672,Everyone believes in something; religion or so...,0
1673,We have only one religion in this world and th...,0
1674,You see what timi Nd his wife is causing... I ...,0
1675,RT @ AMZONE THE DIVINE CURRICULUM. How God's p...,0


In [3]:
data = pd.concat([politics, religion])
data = data.sample(frac=1).reset_index(drop=True)
data.shape

(3354, 2)

In [7]:
0.8*3354

2683.2000000000003

In [8]:
!pip install -q tez transformers

     |████████████████████████████████| 3.3 MB 6.9 MB/s 
     |████████████████████████████████| 895 kB 65.3 MB/s 
     |████████████████████████████████| 3.3 MB 62.3 MB/s 
     |████████████████████████████████| 61 kB 654 kB/s 
     |████████████████████████████████| 596 kB 91.8 MB/s 


In [15]:
import torch
import torch.nn as nn
import transformers


class BERTDataset():
  def __init__(self, data, max_len=512):
    """
      Input:
        texts: Pandas dataframe
        targets: Pandas dataframe
        max_len: (int) maximum number of tokens per block
    """

    self.data = data
    
    self.tokenizer = transformers.BertTokenizer.from_pretrained(
          "bert-base-uncased",
          do_lower_case = False
        )
    self.max_len = max_len

  def __len__(self):
    return len(self.data)

  def __getitem__(self, idx):
    tweet = data.loc[idx]['tweet']
    target = data.loc[idx]['label']

    inputs = self.tokenizer.encode_plus(
          tweet,
          None,
          add_special_tokens = True,
          max_length = self.max_len,
          padding = "max_length",
          truncation = True
        )
    return {
        "ids" : torch.tensor(inputs["input_ids"], dtype = torch.long),
        "mask" : torch.tensor(inputs["attention_mask"], dtype = torch.long),
        "token_type_ids" : torch.tensor(inputs["token_type_ids"], dtype = torch.long),
        "targets" : torch.tensor(target, dtype = torch.float),
        } 

In [11]:
train_no = 2683
train = data.iloc[:train_no]
valid = data.iloc[train_no:].reset_index(drop = True)
train

,tweet,label
0,No sir but reality is reality nCan you ans...,1
1,RT One on One #DebateChallenge n nFollowing...,1
2,RT Our farmers are our pride amp strength ...,1
3,RT One on One #DebateChallenge n nFollowing...,1
4,I fi xe x xa https t co pFD RBj,1
...,...,...
2678,I’m addressing your misrepresentation of who C...,0
2679,India is a civilization. A nation who gloats o...,0
2680,RT Our farmers are our pride amp strength ...,1
2681,But the difference is only Muslims commit rape...,0


In [6]:
valid

,tweet,label
0,RT Modi s corruption is institutional His p...,1
1,xe x x Speed Breaker Didi xe x x tot...,1
2,my bf left his bad religion shirt in my closet...,0
3,Modi Govt Has Spent Rs crore on Ads Sinc...,1
4,RT There is absolutely NOTHING for Modi to s...,1
...,...,...
666,Yes but which religion is making that violence...,0
667,like when u say ur religion is pagan what does...,0
668,RT One on One #DebateChallenge n nFollowing...,1
669,“ Mohammad is Known As The Convener Of The Rel...,0


In [16]:
train_dataset = BERTDataset(train)
valid_dataset = BERTDataset(valid)

In [13]:
import tez
import torch
from transformers import (AdamW,
                          BertModel,
                          get_linear_schedule_with_warmup)
import torch.nn as nn
from sklearn.metrics import hamming_loss

class BERT(tez.Model):
  def __init__(self, no_train_steps, num_classes = 1, lr = 5e-5):
    """
      input:
         num_classes: (int) number of classes
         no_train_steps: (int) number of training steps
         lr: (float) learning rate 
    """
    super().__init__()
    
    self.bert = BertModel.from_pretrained(
            "bert-base-uncased", return_dict = False
            )
    self.lr = lr
    self.dropout = nn.Dropout(0.3)
    self.out = nn.Linear(768, num_classes)


  def fetch_optimizer(self):
    """
        output: (torch.optimizer) A standard pytorch Optimizer
    """
    return AdamW(self.parameters(), self.lr) 
   
  def fetch_scheluder(self):
    """
         output: torch scheduler
    """
    return get_linear_schedule_with_warmup(
            self.optimizer,
            num_warmup_steps = 0,
            num_training_steps = self.no_train_steps
        ) 
  
  def losses(self, out, targets):
    return nn.CrossEntropyLoss()(out, targets.unsqueeze(-1))

  def monitor_metrics(self, outputs, targets):
    y_pred_tag = torch.round(outputs)
    targets = targets.cpu().detach().numpy()
    y_pred_tag = y_pred_tag.cpu().detach().numpy()

    correct_results_sum = (y_pred_tag == targets).sum()
    acc = correct_results_sum/targets.shape[0]
    acc = np.round(acc * 100)
    
    
    
    return {
      "accuracy": acc
    }

  def forward(self, ids, mask, token_type_ids, targets = None):
    _,x = self.bert(ids, attention_mask = mask, token_type_ids = token_type_ids)
    x = self.out(self.dropout(x))

    if targets is not None:
      loss = self.losses(x, targets)
    #   metrics = self.monitor_metrics(x, targets)
      return x, loss, {}
    return x, 0, {}

In [ ]:
bs = 2
epochs = 10

ntrain_steps = int(len(data) / bs * epochs)
model = BERT(no_train_steps = 10)
es = tez.callbacks.EarlyStopping(monitor="valid_loss", patience=3, model_path = 'model.bin')

model.fit(train_dataset, valid_dataset, epochs=epochs, device = "cuda", train_bs = bs, callbacks=[es])